In [2]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

要先建立輸入到模型的資料
- 若是要分類是否是熱點，應該要以一個區域的grid為單位
- 所以建立得grid亦包含該地區的所有特徵資料，以比例顯示

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from utils import get_grid, calculate_gi

from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, average_precision_score

In [5]:
dataA1 = pd.read_csv('../ComputedData/Accident/DataA1_with_MYP.csv')
dataA2 = pd.read_csv('../ComputedData/Accident/DataA2_with_MYP.csv')
taiwan = gpd.read_file('../Data/OFiles_9e222fea-bafb-4436-9b17-10921abc6ef2/TOWN_MOI_1140318.shp')
taiwan = taiwan[(~taiwan['TOWNNAME'].isin(['旗津區', '頭城鎮', '蘭嶼鄉', '綠島鄉', '琉球鄉'])) & 
                (~taiwan['COUNTYNAME'].isin(['金門縣', '連江縣', '澎湖縣']))]

filtered_A2 = dataA2[dataA2['當事者順位'] == 1]
filtered_A1 = dataA1[dataA1['當事者順位'] == 1]

filtered_A1['source'] = 'A1'
filtered_A2['source'] = 'A2'
filtered_A1['num_accidents'] = 1 
filtered_A2['num_accidents'] = 1
combined_data = pd.concat([filtered_A1, filtered_A2], ignore_index=True)

hex_grid = get_grid(combined_data, hex_size=0.01, threshold=-1)
taiwan = taiwan.to_crs(hex_grid.crs)

hex_grid = hex_grid[hex_grid.intersects(taiwan.unary_union)]

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_29915/3893091923.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_A1['source'] = 'A1'
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_29915/3893091923.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_A2['source'] = 'A2'
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_29915/3893091923.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

create hexagon
get grid


/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_29915/3893091923.py:19: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  hex_grid = hex_grid[hex_grid.intersects(taiwan.unary_union)]


hex_grid目前包含所有事故索引，所以要回推
- 回推方式從combined_data獲取，並且計算他們的事故特徵平均

In [12]:
grid = calculate_gi(6, hex_grid, adjacency='knn')
grid = gpd.sjoin(hex_grid, taiwan[['COUNTYNAME', 'geometry']], how='left', predicate='intersects')
grid

geometry  num_accidents  \
67     POLYGON ((152292.359 2551036.405, 151786.254 2...            0.0   
68     POLYGON ((152304.868 2552954.557, 151798.832 2...            1.0   
69     POLYGON ((152317.385 2554872.714, 151811.419 2...            1.0   
70     POLYGON ((152329.911 2556790.874, 151824.015 2...            0.0   
91     POLYGON ((152595.011 2597073.191, 152090.601 2...            0.0   
...                                                  ...            ...   
26179  POLYGON ((350590.702 2765926.598, 350078.927 2...            0.0   
26180  POLYGON ((350576.594 2767845.239, 350064.886 2...            3.0   
26181  POLYGON ((350562.478 2769763.883, 350050.836 2...            0.0   
26379  POLYGON ((352097.818 2766897.135, 351585.969 2...            0.0   
26380  POLYGON ((352083.493 2768815.783, 351571.71 27...            0.0   

              accident_indices  GiZScore          hotspot  index_right  \
67                          [] -0.229282  Not Significant          347   
68                     [61388] -0.229281  Not Significant          347   
69                     [22323] -0.230844  Not Significant          347   
70                          [] -0.230845  Not Significant          347   
91                          [] -0.233970  Not Significant          167   
...                        ...       ...              ...          ...   
26179                       [] -0.226156  Not Significant          303   
26180  [47413, 119090, 219484] -0.232405  Not Significant          303   
26181                       [] -0.226156  Not Significant          303   
26379                       [] -0.227719  Not Significant          303   
26380                       [] -0.227719  Not Significant          303   

      COUNTYNAME  
67           臺南市  
68           臺南市  
69           臺南市  
70           臺南市  
91           雲林縣  
...          ...  
26179        新北市  
26180        新北市  
26181        新北市  
26379        新北市  
26380        新北市  

[18326 rows x 7 columns]

In [57]:
select_group = [
    # 氣候暫不討論
    # '天候名稱', '光線名稱',
    # 道路問題
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    # 車道劃分
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    # 大類別
    '事故類型及型態大類別名稱', '車輛撞擊部位大類別名稱-最初', '車輛撞擊部位大類別名稱-其他',
    '事故位置大類別名稱', '肇因研判大類別名稱-主要', '當事者區分-類別-大類別名稱-車種', '當事者行動狀態大類別名稱',
    '肇因研判大類別名稱-個別', '道路型態大類別名稱',
    # 子類別
    # '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱',
    # '當事者區分-類別-子類別名稱-車種', '肇因研判子類別名稱-主要',
    # '車輛撞擊部位子類別名稱-最初', '當事者行動狀態子類別名稱',
    # '肇因研判子類別名稱-個別', '車輛撞擊部位子類別名稱-其他',
    # '事故位置子類別名稱', '道路型態子類別名稱',
    # 其他
    # '當事者屬-性-別名稱', '當事者事故發生時年齡', '速限-第1當事者', '道路類別-第1當事者-名稱',
    # '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', '肇事逃逸類別名稱-是否肇逃',
    # 設施
    'mrt_100m_count', 'youbike_100m_count', 'parkinglot_100m_count',
    # A1 or A2
    # 'source', 
    ]

In [58]:
def extract_features(
        grid, combined_data, select_group, rows
        ):

    indices = grid['accident_indices'].iloc[rows] # return list of original data index
    sample = combined_data.iloc[indices]
    sample = sample[select_group]

    cat_cols = sample.select_dtypes(include='object').columns
    num_cols = sample.select_dtypes(include='number').columns

    # for categorical features
    cat_features = []
    for col in cat_cols:
        vc = sample[col].value_counts(normalize=True)
        vc.index = [f"{col}_{v}" for v in vc.index]
        cat_features.append(vc)
    cat_features = pd.concat(cat_features)

    # for numerical features
    num_features = sample[num_cols].mean()
    num_features.index = [f"{col}_mean" for col in num_features.index]

    all_features = pd.concat([cat_features, num_features])
    all_features_df = all_features.to_frame().T

    return all_features_df

## Features concat

In [59]:
all_features_list = []
for rows in range(grid.shape[0]):
    features = extract_features(grid, combined_data, select_group, rows)
    all_features_list.append(features)

all_features_df = pd.concat(all_features_list, ignore_index=True)
all_features_df.fillna(0, inplace=True)
all_features_df

mrt_100m_count_mean  youbike_100m_count_mean  \
0                      0.0                      0.0   
1                      0.0                      0.0   
2                      0.0                      0.0   
3                      0.0                      0.0   
4                      0.0                      0.0   
...                    ...                      ...   
18321                  0.0                      0.0   
18322                  0.0                      0.0   
18323                  0.0                      0.0   
18324                  0.0                      0.0   
18325                  0.0                      0.0   

       parkinglot_100m_count_mean  路面狀況-路面鋪裝名稱_柏油  路面狀況-路面狀態名稱_乾燥  \
0                             0.0             0.0             0.0   
1                             0.0             1.0             1.0   
2                             0.0             1.0             1.0   
3                             0.0             0.0             0.0   
4                             0.0             0.0             0.0   
...                           ...             ...             ...   
18321                         0.0             0.0             0.0   
18322                         0.0             1.0             1.0   
18323                         0.0             0.0             0.0   
18324                         0.0             0.0             0.0   
18325                         0.0             0.0             0.0   

       路面狀況-路面缺陷名稱_無缺陷  道路障礙-障礙物名稱_無障礙物  道路障礙-視距品質名稱_無遮蔽物  道路障礙-視距名稱_良好  \
0                  0.0              0.0               0.0           0.0   
1                  1.0              1.0               1.0           1.0   
2                  1.0              1.0               1.0           1.0   
3                  0.0              0.0               0.0           0.0   
4                  0.0              0.0               0.0           0.0   
...                ...              ...               ...           ...   
18321              0.0              0.0               0.0           0.0   
18322              1.0              1.0               1.0           1.0   
18323              0.0              0.0               0.0           0.0   
18324              0.0              0.0               0.0           0.0   
18325              0.0              0.0               0.0           0.0   

       號誌-號誌種類名稱_無號誌  ...  當事者區分-類別-大類別名稱-車種_其他車  肇因研判大類別名稱-主要_無(非車輛駕駛人因素)  \
0           0.000000  ...                    0.0                  0.000000   
1           1.000000  ...                    0.0                  0.000000   
2           1.000000  ...                    0.0                  0.000000   
3           0.000000  ...                    0.0                  0.000000   
4           0.000000  ...                    0.0                  0.000000   
...              ...  ...                    ...                       ...   
18321       0.000000  ...                    0.0                  0.000000   
18322       0.666667  ...                    0.0                  0.333333   
18323       0.000000  ...                    0.0                  0.000000   
18324       0.000000  ...                    0.0                  0.000000   
18325       0.000000  ...                    0.0                  0.000000   

       肇因研判大類別名稱-個別_無(非車輛駕駛人因素)  路面狀況-路面狀態名稱_冰雪  當事者區分-類別-大類別名稱-車種_全聯結車  \
0                      0.000000             0.0                     0.0   
1                      0.000000             0.0                     0.0   
2                      0.000000             0.0                     0.0   
3                      0.000000             0.0                     0.0   
4                      0.000000             0.0                     0.0   
...                         ...             ...                     ...   
18321                  0.000000             0.0                     0.0   
18322                  0.333333             0.0                     0.0   
1832

In [102]:
new_grid = pd.concat([grid.reset_index(drop=True)[['hotspot', 'COUNTYNAME']], all_features_df], axis=1)
new_grid.to_csv("../ComputedData/ForTDA/new_grid.csv", index=False)

In [95]:
# with county town
# 原始資料index並非從1開始所以需reset
new_grid = pd.concat([grid.reset_index(drop=True)[['hotspot', 'COUNTYNAME']], all_features_df], axis=1)
county_dummies = pd.get_dummies(new_grid['COUNTYNAME'], prefix='county')
new_grid_encoded = pd.concat([new_grid.drop(['COUNTYNAME'], axis=1), county_dummies], axis=1)

# new feature
# cond = (
#     (new_grid['mrt_100m_count_mean'] > 0) |
#     (new_grid['youbike_100m_count_mean'] > 0) |
#     (new_grid['parkinglot_100m_count_mean'] > 0)
# )
# new_grid['cross_type'] = cond.astype(int)
# new_grid['cross_type_label'] = new_grid['COUNTYNAME'] + '_' + new_grid['cross_type'].astype(str)
# county_dummies = pd.get_dummies(new_grid['cross_type_label'], prefix='county')
# new_grid_encoded = pd.concat([new_grid.drop(['cross_type_label', 'COUNTYNAME', 'cross_type'], axis=1), county_dummies], axis=1)

# without county town
# new_grid = pd.concat([grid.reset_index(drop=True)[['hotspot']], all_features_df], axis=1)
# new_grid_encoded = new_grid

# binary hotspot
new_grid_encoded['hotspot'] = new_grid_encoded['hotspot'].apply(lambda x: 'Hotspot' if 'Hotspot' in str(x) else 'Not Hotspot')

le = LabelEncoder()
y = le.fit_transform(new_grid_encoded['hotspot'])
X = new_grid_encoded.drop(columns=['hotspot'])

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
y_train = pd.Series(y_train, index=X_train.index)
y_test  = pd.Series(y_test,  index=X_test.index)

# with undersampling
cls_counts = y_test.value_counts()
min_count = cls_counts.min()
rus_test = RandomUnderSampler(
    sampling_strategy={int(c): int(min_count) for c in cls_counts.index},
    random_state=42
)
X_resampled_test, y_resampled_test = rus_test.fit_resample(X_test, y_test)

# without undersampling
# X_resampled_test = X_test.copy()
# y_resampled_test = y_test.copy()

print("before US")
print(pd.Series(y_test).map(dict(enumerate(le.classes_))).value_counts())
print("after US")
print(pd.Series(y_resampled_test).map(dict(enumerate(le.classes_))).value_counts())

before US
Not Hotspot    5259
Hotspot         239
Name: count, dtype: int64
after US
Hotspot        239
Not Hotspot    239
Name: count, dtype: int64


/Users/wangqiqian/opt/anaconda3/envs/ST-RTA/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/wangqiqian/opt/anaconda3/envs/ST-RTA/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [88]:
lr = LogisticRegression(
        class_weight='balanced', max_iter=1000, 
        random_state=42, 
        # multi_class='multinomial'
    )
rf = RandomForestClassifier(
        n_estimators=300, max_depth=None, min_samples_leaf=1,
        class_weight='balanced', n_jobs=-1, random_state=42,
    )

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, clf in [('Logistic', lr), ('RandomForest', rf)]:
    scores = cross_val_score(clf, X_train, y_train, cv=cv, n_jobs=-1,
                            #  scoring='roc_auc_ovr_weighted',
                            scoring='roc_auc'
                             )
    print(f'{name} CV ROC AUC: {scores.mean():.3f} ± {scores.std():.3f}')

rf.fit(X_train, y_train)

proba_test_bal = rf.predict_proba(X_resampled_test)
y_pred_bal = np.argmax(proba_test_bal, axis=1)

Logistic CV ROC AUC: 0.987 ± 0.002
RandomForest CV ROC AUC: 0.995 ± 0.003


In [90]:
from sklearn.preprocessing import label_binarize

print("\nConfusion Matrix")
print(confusion_matrix(y_resampled_test, y_pred_bal, labels=range(len(le.classes_))))

print("\nClassification Report")
print(classification_report(
    y_resampled_test, y_pred_bal, target_names=le.classes_, digits=3
))


if proba_test_bal.shape[1] == 2:
    # 二元分類
    roc_auc = roc_auc_score(y_resampled_test, proba_test_bal[:, 1])
    print(f'ROC AUC: {roc_auc:.3f}')
    # 多類PR AUC需要 binarize 後用 one-vs-rest，再做 macro/weighted 平均
    y_test_bin = label_binarize(y_resampled_test, classes=range(len(le.classes_)))  # shape [n, n_classes]
    pr_auc_macro  = average_precision_score(y_test_bin, proba_test_bal[:, 1], average='macro')
    pr_auc_weight = average_precision_score(y_test_bin, proba_test_bal[:, 1], average='weighted')
    print(f'PR  AUC macro: {pr_auc_macro:.3f}')
    print(f'PR  AUC wighted: {pr_auc_weight:.3f}')
else:
    # 多類分類
    roc_auc = roc_auc_score(y_resampled_test, proba_test_bal, average='weighted', multi_class='ovr')
    print(f'ROC AUC: {roc_auc:.3f}')
    # 多類PR AUC需要 binarize 後用 one-vs-rest，再做 macro/weighted 平均
    y_test_bin = label_binarize(y_resampled_test, classes=range(len(le.classes_)))  # shape [n, n_classes]
    pr_auc_macro  = average_precision_score(y_test_bin, proba_test_bal, average='macro')
    pr_auc_weight = average_precision_score(y_test_bin, proba_test_bal, average='weighted')
    print(f'PR  AUC macro: {pr_auc_macro:.3f}')
    print(f'PR  AUC wighted: {pr_auc_weight:.3f}')


Confusion Matrix
[[208  31]
 [  1 238]]

Classification Report
              precision    recall  f1-score   support

     Hotspot      0.995     0.870     0.929       239
 Not Hotspot      0.885     0.996     0.937       239

    accuracy                          0.933       478
   macro avg      0.940     0.933     0.933       478
weighted avg      0.940     0.933     0.933       478

ROC AUC: 0.993
PR  AUC macro: 0.988
PR  AUC wighted: 0.988


設施平均：該地區的事故點附近平均會有幾個設施

In [91]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X_train.columns

for i in indices:
    print(f"{feature_names[i]}: {importances[i]:.4f}")

youbike_100m_count_mean: 0.1033
號誌-號誌種類名稱_行車管制號誌(附設行人專用號誌): 0.0822
當事者區分-類別-大類別名稱-車種_慢車: 0.0578
當事者行動狀態大類別名稱_人的狀態: 0.0541
事故類型及型態大類別名稱_人與車: 0.0489
當事者區分-類別-大類別名稱-車種_人: 0.0461
肇因研判大類別名稱-主要_非駕駛者: 0.0396
肇因研判大類別名稱-個別_非駕駛者: 0.0375
車道劃分設施-分道設施-快車道或一般車道間名稱_禁止變換車道線(無標記): 0.0346
事故位置大類別名稱_其他: 0.0317
車道劃分設施-分道設施-快慢車道間名稱_快慢車道分隔線: 0.0290
車輛撞擊部位大類別名稱-最初_其他: 0.0268
車道劃分設施-分道設施-快車道或一般車道間名稱_車道線(附標記): 0.0237
事故類型及型態大類別名稱_車與車: 0.0215
號誌-號誌動作名稱_正常: 0.0210
事故類型及型態大類別名稱_車輛本身: 0.0171
車道劃分設施-分向設施大類別名稱_中央分向島: 0.0171
道路障礙-視距品質名稱_有遮蔽物: 0.0164
號誌-號誌種類名稱_行車管制號誌: 0.0156
車道劃分設施-分向設施大類別名稱_行車分向線: 0.0147
號誌-號誌種類名稱_閃光號誌: 0.0141
車道劃分設施-分道設施-路面邊線名稱_無: 0.0123
肇因研判大類別名稱-個別_無(車輛駕駛者因素): 0.0114
路面狀況-路面狀態名稱_濕潤: 0.0101
肇因研判大類別名稱-個別_其他: 0.0098
當事者行動狀態大類別名稱_車的狀態: 0.0094
車道劃分設施-分道設施-快車道或一般車道間名稱_車道線(無標記): 0.0088
肇因研判大類別名稱-主要_其他: 0.0081
當事者區分-類別-大類別名稱-車種_小客車(含客、貨兩用): 0.0072
道路障礙-障礙物名稱_無障礙物: 0.0066
當事者區分-類別-大類別名稱-車種_小貨車: 0.0065
事故位置大類別名稱_交叉路口: 0.0064
肇因研判大類別名稱-主要_無(車輛駕駛者因素): 0.0062
當事者區分-類別-大類別名稱-車種_大客車: 0.0054
道路障礙-障礙物名稱_其他障礙物: 0.0